In [ ]:
from pulp import *
import time

import numpy as np
from random import randint
import matplotlib.pyplot as matp


#create 30 insances to test
iter = 0
while iter < 30:
    prob = pulp.LpProblem("Transportation Problem", pulp.LpMinimize)

    #Supply locations, X, from 1 to N
    N = 400
    Supply = {}
    for s in range (0,N):
        Supply[s] = randint(1,100)


    #demand locations, Y from 1 to M
    M = 400
    Demand = {}
    for d in range (0,M-1):
        Demand[d] = randint(1,100)

 
    FIX = sum(Supply.values())-sum(Demand.values())

    if FIX > 0:
        Demand[M-1] = FIX
    elif FIX < 0:
        Supply[N-1] = Supply[N-1]-FIX
    else:
        Demand[M-1] = 0


    #create matrix with arc weights
    ARC = np.random.uniform(1,9,size=(N,M))


    #create ARC connections between supply and demand nodes
    Routes = {}
    Routes = [(n,m) for n in range(0,N) for m in range(0,M)]


    Route_vars = LpVariable.dicts("X",(Routes),0,None,LpInteger)

    for s in Supply:
        prob += lpSum([Route_vars[s,m] for m in Demand]) <= Supply[s]

    for d in Demand:
        prob += lpSum([Route_vars[n,d] for n in Supply]) >= Demand[d]

    #solvers
    name = {1:"GLPK",2:"CPLEX",3:"GUROBI",4:"CBC"}

    #CBC   
    start_time = time.time()
    prob.solve()

    CBCtime = {}
    CBCtime[iter] = (time.time() - start_time)
    CBC_Status = LpStatus[prob.status]


    #GUROBI
    start_time = time.time()
    prob.solve(GUROBI(msg=0))
    GUROBItime = {}
    GUROBItime[iter] = (time.time() - start_time)


    #GLPK
    start_time = time.time()
    prob.solve(GLPK(msg=0))
    GLPKtime = {}
    GLPKtime[iter] = (time.time() - start_time)



    start_time = time.time()
    prob.solve(CPLEX(msg=0))
    CPLEXtime = {}
    CPLEXtime[iter] = (time.time() - start_time)


    if CBC_Status == "Optimal":
        print "Optimal status, store results"
        iter = iter + 1
        print("------------------------ %s Iteration -----------------------" % iter)
    else:
        print "infeasable, run another........................"

#graph average result of 30 successful trials
CBC_Graph = (sum(CBCtime.values())/30)  
Gurobi_Graph = (sum(GUROBItime.values())/30) 
GLPK_Graph = (sum(GLPKtime.values())/30) 
CPLEX_Graph = (sum(CPLEXtime.values())/30)
Times = (CBC_Graph, Gurobi_Graph, GLPK_Graph, CPLEX_Graph)
Index = np.arange(4)


matp.bar(Index, Times)
matp.xticks(Index, ("CBC/Coin", "Gurobi", "GLPK", "CPLEX"))
matp.ylabel('Time in seconds')
matp.show()

